In [ ]:
import os
import numpy as np
from pyodesys.symbolic import SymbolicSys, ScaledSys
from pyodesys.native import native_sys
from pyodesys.tests._robertson import get_ode_exprs
print({k: os.environ[k] for k in 'OMP_NUM_THREADS ANYODE_NUM_THREADS'.split()})

In [ ]:
logc, logt, reduced, powsimp = False, False, False, False

In [ ]:
odesys = SymbolicSys.from_callback(
    get_ode_exprs(logc, logt, reduced)[0],
    2 if reduced else 3, 6 if reduced else 3, names='ABC')

In [ ]:
native = native_sys['cvode'].from_other(odesys)

In [ ]:
N = 500
tout = np.array([[0, 1e11]]*N)
y0 = np.array([[1,0,0]]*N)
params = [0.04, 1e4, 3e7]

In [ ]:
results = native.integrate(tout, y0, params, nsteps=5000)
assert len(results) == N
import time
t0 = time.time()
results = native.integrate(tout, y0, params, nsteps=5000)
print(time.time() - t0)

In [ ]:
dep_scaling, indep_scaling = 1e6, 1
scaledsys = ScaledSys.from_other(odesys, dep_scaling=dep_scaling, indep_scaling=indep_scaling, exprs_process_cb=(
    lambda exprs: [sp.powsimp(expr.expand(), force=True) for expr in exprs]) if powsimp else None)
scalednative = native_sys['cvode'].from_other(scaledsys)

In [ ]:
%prun native.integrate(tout, y0, params, nsteps=5000)

In [ ]:
results = scalednative.integrate(tout, y0, params, nsteps=5000)
assert len(results) == N
import time
t0 = time.time()
results = scalednative.integrate(tout, y0, params, nsteps=5000)
print(time.time() - t0)

In [ ]:
%prun native.integrate(tout, y0, params, nsteps=5000)